# Level 3

Level 3 represents the highest level of user control and customization in Aviary's user interface.
At this level, users have full access to Python and OpenMDAO methods that Aviary calls.
They can use the complete set of Aviary's methods, functionalities, and classes to construct and fine-tune their aircraft models.
Level 3 enables users to have supreme control over every aspect of the model, including subsystems, connections, and advanced optimization techniques.

Level 3 is the most complex but specific methods defined at this level are used in levels 1 and 2, hopefully reducing user activation energy when learning more about Aviary.
This progressive approach helps users gradually enhance their analysis capabilities and adapt to more complex modeling requirements as they gain proficiency and experience.

We will show two approaches of level 3 development using two examples: The first one will implement all methods differently from what are available in `aviary/interface/methods_for_level2.py` but will follow the same steps. It will show that you don't have to have a `.csv` input file and you don't have to have a `phase_info` dictionary. The second is a `solved` mission approach that is embedded into all the methods of `aviary/interface/methods_for_level2.py`.

```{note}
For each of these examples we have set `max_iter = 0`, which means that the optimization will not run. This is done to reduce the computational time for the examples. If you want to run the optimization, you can set `max_iter = 100` or some similar value.
```

## A level 3 example: N3CC

In [level 2](onboarding_level2.ipynb), we have shown how to follow the standard steps to build an Aviary model, but sometimes you may find that the model you have in mind does not match that predefined structure. If you start a new model that cannot been embedded in those methods, we can write special methods ourselves. This example will show you how to do that.

In `aviary/validation_cases/benchmark_tests` folder, there is an N3CC aircraft full mission benchmark test `test_FLOPS_based_full_mission_N3CC.py`. Now, we will show how to create an N3CC example in level3. The key is that we follow the same steps:
- init
- load_inputs
- check_inputs (optional)
- add_pre_mission_systems
- add_phases
- add_post_mission_systems
- link_phases
- add_driver
- add_design_variables (optional)
- add_objective
- setup
- set_initial_guesses
- run_aviary_problem

### Examining an N3CC design case

In [ ]:
'''
NOTES:
Includes:
Takeoff, Climb, Cruise, Descent, Landing
Computed Aero
N3CC data
'''
import warnings

import dymos as dm
import openmdao.api as om
import scipy.constants as _units
from packaging import version

from aviary.api import Aircraft, Mission, Dynamic
import aviary.api as av

use_new_dymos_syntax = version.parse(dm.__version__) > version.parse("1.8.0")

# benchmark based on N3CC (fixed cruise alt) FLOPS model


##########################################
# init problem                           #
##########################################
prob = om.Problem(model=om.Group())

prob.driver = driver = om.pyOptSparseDriver(optimizer='IPOPT')
driver.opt_settings["max_iter"] = 0
driver.opt_settings["tol"] = 1e-3
driver.opt_settings['print_level'] = 4

##########################################
# Aircraft Input Variables and Options   #
##########################################

aviary_inputs = av.get_flops_inputs('N3CC')

aviary_inputs.set_val(
    Mission.Landing.LIFT_COEFFICIENT_MAX, 2.4, units='unitless')
aviary_inputs.set_val(
    Mission.Takeoff.LIFT_COEFFICIENT_MAX, 2.0, units='unitless')
aviary_inputs.set_val(
    Mission.Takeoff.ROLLING_FRICTION_COEFFICIENT, val=.0175,
    units='unitless')

takeoff_fuel_burned = 577  # lbm
takeoff_thrust_per_eng = 24555.5  # lbf
takeoff_L_over_D = 17.35

aviary_inputs.set_val(
    Mission.Takeoff.FUEL_SIMPLE, takeoff_fuel_burned, units='lbm')
aviary_inputs.set_val(
    Mission.Takeoff.LIFT_OVER_DRAG, takeoff_L_over_D, units='unitless')
aviary_inputs.set_val(
    Mission.Design.THRUST_TAKEOFF_PER_ENG, takeoff_thrust_per_eng, units='lbf')

##########################################
# Phase Info                             #
##########################################

alt_airport = 0  # ft
cruise_mach = .785

alt_i_climb = 0*_units.foot  # m
alt_f_climb = 35000.0*_units.foot  # m
mass_i_climb = 131000*_units.lb  # kg
mass_f_climb = 126000*_units.lb  # kg
v_i_climb = 198.44*_units.knot  # m/s
v_f_climb = 455.49*_units.knot  # m/s
# initial mach set to lower value so it can intersect with takeoff end mach
# mach_i_climb = 0.3
mach_i_climb = 0.2
mach_f_climb = cruise_mach
range_i_climb = 0*_units.nautical_mile  # m
range_f_climb = 160.3*_units.nautical_mile  # m
t_i_climb = 2 * _units.minute  # sec
t_f_climb = 26.20*_units.minute  # sec
t_duration_climb = t_f_climb - t_i_climb

alt_i_cruise = 35000*_units.foot  # m
alt_f_cruise = 35000*_units.foot  # m
alt_min_cruise = 35000*_units.foot  # m
alt_max_cruise = 35000*_units.foot  # m
mass_i_cruise = 126000*_units.lb  # kg
mass_f_cruise = 102000*_units.lb  # kg
v_i_cruise = 455.49*_units.knot  # m/s
v_f_cruise = 455.49*_units.knot  # m/s
mach_min_cruise = cruise_mach
mach_max_cruise = cruise_mach
range_i_cruise = 160.3*_units.nautical_mile  # m
range_f_cruise = 3243.9*_units.nautical_mile  # m
t_i_cruise = 26.20*_units.minute  # sec
t_f_cruise = 432.38*_units.minute  # sec
t_duration_cruise = t_f_cruise - t_i_cruise

alt_i_descent = 35000*_units.foot
# final altitude set to 35 to ensure landing is feasible point
# alt_f_descent = 0*_units.foot
alt_f_descent = 35*_units.foot
v_i_descent = 455.49*_units.knot
v_f_descent = 198.44*_units.knot
mach_i_descent = cruise_mach
mach_f_descent = 0.3
mass_i_descent = 102000*_units.pound
mass_f_descent = 101000*_units.pound
range_i_descent = 3243.9*_units.nautical_mile
range_f_descent = 3378.7*_units.nautical_mile
t_i_descent = 432.38*_units.minute
t_f_descent = 461.62*_units.minute
t_duration_descent = t_f_descent - t_i_descent

##################
# Define Phases  #
##################

num_segments_climb = 6
num_segments_cruise = 1
num_segments_descent = 5

climb_seg_ends, _ = dm.utils.lgl.lgl(num_segments_climb + 1)
descent_seg_ends, _ = dm.utils.lgl.lgl(num_segments_descent + 1)

transcription_climb = dm.Radau(
    num_segments=num_segments_climb, order=3, compressed=True,
    segment_ends=climb_seg_ends)
transcription_cruise = dm.Radau(
    num_segments=num_segments_cruise, order=3, compressed=True)
transcription_descent = dm.Radau(
    num_segments=num_segments_descent, order=3, compressed=True,
    segment_ends=descent_seg_ends)

takeoff_options = av.HeightEnergyTakeoffPhaseBuilder(
    airport_altitude=alt_airport,  # ft
    num_engines=aviary_inputs.get_val(Aircraft.Engine.NUM_ENGINES)
)

climb_options = av.HeightEnergyClimbPhaseBuilder(
    'test_climb',
    user_options=av.AviaryValues({
        'initial_altitude': (alt_i_climb, 'm'),
        'final_altitude': (alt_f_climb, 'm'),
        'initial_mach': (mach_i_climb, 'unitless'),
        'final_mach': (mach_f_climb, 'unitless'),
        'fix_initial': (False, 'unitless'),
        'fix_range': (False, 'unitless'),
        'input_initial': (True, 'unitless'),
    }),
    core_subsystems=av.default_mission_subsystems,
    subsystem_options={'core_aerodynamics': {'method': 'computed'}},
    transcription=transcription_climb,
)

cruise_options = av.HeightEnergyCruisePhaseBuilder(
    'test_cruise',
    user_options=av.AviaryValues({
        'min_altitude': (alt_min_cruise, 'm'),
        'max_altitude': (alt_max_cruise, 'm'),
        'min_mach': (mach_min_cruise, 'unitless'),
        'max_mach': (mach_max_cruise, 'unitless'),
        'required_available_climb_rate': (300, 'ft/min'),
        'fix_initial': (False, 'unitless'),
        'fix_final': (False, 'unitless'),
    }),
    core_subsystems=av.default_mission_subsystems,
    subsystem_options={'core_aerodynamics': {'method': 'computed'}},
    transcription=transcription_cruise,
)

descent_options = av.HeightEnergyDescentPhaseBuilder(
    'test_descent',
    user_options=av.AviaryValues({
        'final_altitude': (alt_f_descent, 'm'),
        'initial_altitude': (alt_i_descent, 'm'),
        'initial_mach': (mach_i_descent, 'unitless'),
        'final_mach': (mach_f_descent, 'unitless'),
        'fix_initial': (False, 'unitless'),
        'fix_range': (True, 'unitless'),
    }),
    core_subsystems=av.default_mission_subsystems,
    subsystem_options={'core_aerodynamics': {'method': 'computed'}},
    transcription=transcription_descent,
)

landing_options = av.HeightEnergyLandingPhaseBuilder(
    ref_wing_area=aviary_inputs.get_val(Aircraft.Wing.AREA, 'ft**2'),
    Cl_max_ldg=aviary_inputs.get_val(Mission.Landing.LIFT_COEFFICIENT_MAX)
)

av.preprocess_crewpayload(aviary_inputs)

##########################################
# add pre mission systems                #
##########################################

# Upstream static analysis for aero
prob.model.add_subsystem(
    'pre_mission',
    av.CorePreMission(aviary_options=aviary_inputs,
                    subsystems=av.default_premission_subsystems),
    promotes_inputs=['aircraft:*', 'mission:*'],
    promotes_outputs=['aircraft:*', 'mission:*'])

##########################################
# add phases                             #
##########################################

# directly connect phases (strong_couple = True), or use linkage constraints (weak
# coupling / strong_couple=False)
strong_couple = False

takeoff = takeoff_options.build_phase(False)

climb = climb_options.build_phase(aviary_options=aviary_inputs)

cruise = cruise_options.build_phase(aviary_options=aviary_inputs)

descent = descent_options.build_phase(aviary_options=aviary_inputs)

landing = landing_options.build_phase(False)

prob.model.add_subsystem(
    'takeoff', takeoff, promotes_inputs=['aircraft:*', 'mission:*'],
    promotes_outputs=['mission:*'])

traj = prob.model.add_subsystem('traj', dm.Trajectory())

# if fix_initial is false, can we always set input_initial to be true for
# necessary states, and then ignore if we use a linkage?
climb.set_time_options(
    fix_initial=True, fix_duration=False, units='s',
    duration_bounds=(1, t_duration_climb*2), duration_ref=t_duration_climb)
cruise.set_time_options(
    fix_initial=False, fix_duration=False, units='s',
    duration_bounds=(1, t_duration_cruise*2), duration_ref=t_duration_cruise)
descent.set_time_options(
    fix_initial=False, fix_duration=False, units='s',
    duration_bounds=(1, t_duration_descent*2), duration_ref=t_duration_descent)

traj.add_phase('climb', climb)

traj.add_phase('cruise', cruise)

traj.add_phase('descent', descent)

if use_new_dymos_syntax:
    climb.timeseries_options['use_prefix'] = True
    cruise.timeseries_options['use_prefix'] = True
    descent.timeseries_options['use_prefix'] = True

##########################################
# add post mission system                #
##########################################

prob.model.add_subsystem(
    'landing', landing, promotes_inputs=['aircraft:*', 'mission:*'],
    promotes_outputs=['mission:*'])

##########################################
# link phases                            #
##########################################

traj.link_phases(["climb", "cruise"], ["time", Dynamic.Mission.ALTITUDE,
                    Dynamic.Mission.VELOCITY, Dynamic.Mission.MASS, Dynamic.Mission.RANGE], connected=strong_couple)
traj.link_phases(["cruise", "descent"], ["time", Dynamic.Mission.ALTITUDE,
                    Dynamic.Mission.VELOCITY, Dynamic.Mission.MASS, Dynamic.Mission.RANGE], connected=strong_couple)

traj = av.setup_trajectory_params(prob.model, traj, aviary_inputs)

##################################
# Connect in Takeoff and Landing #
##################################

prob.model.add_subsystem(
    "takeoff_constraints",
    om.ExecComp(
        [
            "takeoff_mass_con=takeoff_mass-climb_start_mass",
            "takeoff_range_con=takeoff_range-climb_start_range",
            "takeoff_vel_con=takeoff_vel-climb_start_vel",
            "takeoff_alt_con=takeoff_alt-climb_start_alt"
        ],
        takeoff_mass_con={'units': 'lbm'}, takeoff_mass={'units': 'lbm'},
        climb_start_mass={'units': 'lbm'},
        takeoff_range_con={'units': 'ft'}, takeoff_range={'units': 'ft'},
        climb_start_range={'units': 'ft'},
        takeoff_vel_con={'units': 'm/s'}, takeoff_vel={'units': 'm/s'},
        climb_start_vel={'units': 'm/s'},
        takeoff_alt_con={'units': 'ft'}, takeoff_alt={'units': 'ft'},
        climb_start_alt={'units': 'ft'}
    ),
    promotes_inputs=[
        ("takeoff_mass", Mission.Takeoff.FINAL_MASS),
        ("takeoff_range", Mission.Takeoff.GROUND_DISTANCE),
        ("takeoff_vel", Mission.Takeoff.FINAL_VELOCITY),
        ("takeoff_alt", Mission.Takeoff.FINAL_ALTITUDE),
    ],
    promotes_outputs=["takeoff_mass_con", "takeoff_range_con",
                        "takeoff_vel_con", "takeoff_alt_con"],
)

prob.model.connect('traj.climb.states:mass',
                    'takeoff_constraints.climb_start_mass', src_indices=[0])
prob.model.connect('traj.climb.states:range',
                    'takeoff_constraints.climb_start_range', src_indices=[0])
prob.model.connect('traj.climb.states:velocity',
                    'takeoff_constraints.climb_start_vel', src_indices=[0])
prob.model.connect('traj.climb.states:altitude',
                    'takeoff_constraints.climb_start_alt', src_indices=[0])

prob.model.connect(Mission.Takeoff.FINAL_MASS,
                    'traj.climb.initial_states:mass')
prob.model.connect(Mission.Takeoff.GROUND_DISTANCE,
                    'traj.climb.initial_states:range')
prob.model.connect(Mission.Takeoff.FINAL_VELOCITY,
                    'traj.climb.initial_states:velocity')
prob.model.connect(Mission.Takeoff.FINAL_ALTITUDE,
                    'traj.climb.initial_states:altitude')

prob.model.connect('traj.descent.states:mass',
                    Mission.Landing.TOUCHDOWN_MASS, src_indices=[-1])
prob.model.connect('traj.descent.states:altitude', Mission.Landing.INITIAL_ALTITUDE,
                    src_indices=[-1])

##########################
# Add Objective Function #
##########################

# This is an example of a overall mission objective
# create a compound objective that minimizes climb time and maximizes final mass
# we are maxing final mass b/c we don't have an independent value for fuel_mass yet
# we are going to normalize these (makign each of the sub-objectives approx = 1 )
prob.model.add_subsystem(
    "regularization",
    om.ExecComp(
        "reg_objective = - descent_mass_final/60000",
        reg_objective=0.0,
        descent_mass_final={"units": "kg", "shape": 1},
    ),
    promotes_outputs=['reg_objective']
)
# connect the final mass from cruise into the objective
prob.model.connect("traj.descent.states:mass",
                    "regularization.descent_mass_final", src_indices=[-1])

prob.model.add_objective('reg_objective', ref=1)

prob.model.add_subsystem(
    'input_sink',
    av.VariablesIn(aviary_options=aviary_inputs),
    promotes_inputs=['*'],
    promotes_outputs=['*']
)

# suppress warnings:
# "input variable '...' promoted using '*' was already promoted using 'aircraft:*'
with warnings.catch_warnings():

    # Set initial default values for all LEAPS aircraft variables.
    warnings.simplefilter("ignore", om.PromotionWarning)
    av.set_aviary_initial_values(prob.model, aviary_inputs)

    warnings.simplefilter("ignore", om.PromotionWarning)
    prob.setup()

###########################################
# Intial Settings for States and Controls #
###########################################

prob.set_val('traj.climb.t_initial', t_i_climb, units='s')
prob.set_val('traj.climb.t_duration', t_duration_climb, units='s')

prob.set_val('traj.climb.states:altitude', climb.interp(
    Dynamic.Mission.ALTITUDE, ys=[alt_i_climb, alt_f_climb]), units='m')
# prob.set_val(
#     'traj.climb.states:velocity', climb.interp(Dynamic.Mission.VELOCITY, ys=[170, v_f_climb]),
#     units='m/s')
prob.set_val('traj.climb.states:velocity', climb.interp(
    Dynamic.Mission.VELOCITY, ys=[v_i_climb, v_f_climb]), units='m/s')
prob.set_val('traj.climb.states:mass', climb.interp(
    Dynamic.Mission.MASS, ys=[mass_i_climb, mass_f_climb]), units='kg')
prob.set_val('traj.climb.states:range', climb.interp(
    Dynamic.Mission.RANGE, ys=[range_i_climb, range_f_climb]), units='m')  # nmi

prob.set_val('traj.climb.controls:velocity_rate',
                climb.interp(Dynamic.Mission.VELOCITY_RATE, ys=[0.25, 0.0]),
                units='m/s**2')
prob.set_val('traj.climb.controls:throttle',
                climb.interp(Dynamic.Mission.THROTTLE, ys=[1.0, 1.0]),
                units='unitless')

prob.set_val('traj.cruise.t_initial', t_i_cruise, units='s')
prob.set_val('traj.cruise.t_duration', t_duration_cruise, units='s')

prob.set_val('traj.cruise.states:altitude', cruise.interp(
    Dynamic.Mission.ALTITUDE, ys=[alt_i_cruise, alt_f_cruise]), units='m')
prob.set_val('traj.cruise.states:velocity', cruise.interp(
    Dynamic.Mission.VELOCITY, ys=[v_i_cruise, v_f_cruise]), units='m/s')
prob.set_val('traj.cruise.states:mass', cruise.interp(
    Dynamic.Mission.MASS, ys=[mass_i_cruise, mass_f_cruise]), units='kg')
prob.set_val('traj.cruise.states:range', cruise.interp(
    Dynamic.Mission.RANGE, ys=[range_i_cruise, range_f_cruise]), units='m')  # nmi

prob.set_val('traj.cruise.controls:velocity_rate',
                cruise.interp(Dynamic.Mission.VELOCITY_RATE, ys=[0.0, 0.0]),
                units='m/s**2')
prob.set_val('traj.cruise.controls:throttle',
                cruise.interp(Dynamic.Mission.THROTTLE, ys=[0.8, 0.75]),
                units='unitless')

prob.set_val('traj.descent.t_initial', t_i_descent, units='s')
prob.set_val('traj.descent.t_duration', t_duration_descent, units='s')

prob.set_val('traj.descent.states:altitude', descent.interp(
    Dynamic.Mission.ALTITUDE, ys=[alt_i_descent, alt_f_descent]), units='m')
prob.set_val('traj.descent.states:velocity', descent.interp(
    Dynamic.Mission.VELOCITY, ys=[v_i_descent, v_f_descent]), units='m/s')
prob.set_val('traj.descent.states:mass', descent.interp(
    Dynamic.Mission.MASS, ys=[mass_i_descent, mass_f_descent]), units='kg')
prob.set_val('traj.descent.states:range', descent.interp(
    Dynamic.Mission.RANGE, ys=[range_i_descent, range_f_descent]), units='m')

prob.set_val('traj.descent.controls:velocity_rate',
                descent.interp(Dynamic.Mission.VELOCITY_RATE, ys=[-0.25, 0.0]),
                units='m/s**2')
prob.set_val('traj.descent.controls:throttle',
                descent.interp(Dynamic.Mission.THROTTLE, ys=[0.0, 0.0]),
                units='unitless')

# Turn off solver printing so that the SNOPT output is readable.
prob.set_solver_print(level=0)

dm.run_problem(prob, make_plots=False, solution_record_file='N3CC_full_mission.db')
prob.record("final")
prob.cleanup()

times_climb = prob.get_val('traj.climb.timeseries.time', units='s')
altitudes_climb = prob.get_val(
    'traj.climb.timeseries.states:altitude', units='m')
masses_climb = prob.get_val('traj.climb.timeseries.states:mass', units='kg')
ranges_climb = prob.get_val('traj.climb.timeseries.states:range', units='m')
velocities_climb = prob.get_val(
    'traj.climb.timeseries.states:velocity', units='m/s')
thrusts_climb = prob.get_val('traj.climb.timeseries.thrust_net_total', units='N')
times_cruise = prob.get_val('traj.cruise.timeseries.time', units='s')
altitudes_cruise = prob.get_val(
    'traj.cruise.timeseries.states:altitude', units='m')
masses_cruise = prob.get_val('traj.cruise.timeseries.states:mass', units='kg')
ranges_cruise = prob.get_val('traj.cruise.timeseries.states:range', units='m')
velocities_cruise = prob.get_val(
    'traj.cruise.timeseries.states:velocity', units='m/s')
thrusts_cruise = prob.get_val(
    'traj.cruise.timeseries.thrust_net_total', units='N')
times_descent = prob.get_val('traj.descent.timeseries.time', units='s')
altitudes_descent = prob.get_val(
    'traj.descent.timeseries.states:altitude', units='m')
masses_descent = prob.get_val('traj.descent.timeseries.states:mass', units='kg')
ranges_descent = prob.get_val('traj.descent.timeseries.states:range', units='m')
velocities_descent = prob.get_val(
    'traj.descent.timeseries.states:velocity', units='m/s')
thrusts_descent = prob.get_val(
    'traj.descent.timeseries.thrust_net_total', units='N')

print("-------------------------------")
print(f"times_climb: {times_climb[-1]} (s)")
print(f"altitudes_climb: {altitudes_climb[-1]} (m)")
print(f"masses_climb: {masses_climb[-1]} (kg)")
print(f"ranges_climb: {ranges_climb[-1]} (m)")
print(f"velocities_climb: {velocities_climb[-1]} (m/s)")
print(f"thrusts_climb: {thrusts_climb[-1]} (N)")
print(f"times_cruise: {times_cruise[-1]} (s)")
print(f"altitudes_cruise: {altitudes_cruise[-1]} (m)")
print(f"masses_cruise: {masses_cruise[-1]} (kg)")
print(f"ranges_cruise: {ranges_cruise[-1]} (m)")
print(f"velocities_cruise: {velocities_cruise[-1]} (m/s)")
print(f"thrusts_cruise: {thrusts_cruise[-1]} (N)")
print(f"times_descent: {times_descent[-1]} (s)")
print(f"altitudes_descent: {altitudes_descent[-1]} (m)")
print(f"masses_descent: {masses_descent[-1]} (kg)")
print(f"ranges_descent: {ranges_descent[-1]} (m)")
print(f"velocities_descent: {velocities_descent[-1]} (m/s)")
print(f"thrusts_descent: {thrusts_descent[-1]} (N)")
print("-------------------------------")


### Move each code block to a function

This code is quite verbose!

To make it easier to follow the steps mentioned above, we split this script into several functions each representing a step at the beginning of this section (except `check_inputs()` and `add_design_variables()`).
These methods also closely mirror those from `aviary/interface/methods_for_level2.py`, so you can draw parallels between Level 2 and 3.

Here's what the new run script looks like:

In [ ]:
'''
NOTES:
Includes:
Takeoff, Climb, Cruise, Descent, Landing
Computed Aero
N3CC data
'''
import warnings

import dymos as dm
import openmdao.api as om
import scipy.constants as _units
from packaging import version

import aviary.api as av

use_new_dymos_syntax = version.parse(dm.__version__) > version.parse("1.8.0")


##########################################
# Phase Info
##########################################

alt_airport = 0  # ft
cruise_mach = .785

alt_i_climb = 0*_units.foot  # m
alt_f_climb = 35000.0*_units.foot  # m
mass_i_climb = 131000*_units.lb  # kg
mass_f_climb = 126000*_units.lb  # kg
v_i_climb = 198.44*_units.knot  # m/s
v_f_climb = 455.49*_units.knot  # m/s
# initial mach set to lower value so it can intersect with takeoff end mach
# mach_i_climb = 0.3
mach_i_climb = 0.2
mach_f_climb = cruise_mach
range_i_climb = 0*_units.nautical_mile  # m
range_f_climb = 160.3*_units.nautical_mile  # m
t_i_climb = 2 * _units.minute  # sec
t_f_climb = 26.20*_units.minute  # sec
t_duration_climb = t_f_climb - t_i_climb

alt_i_cruise = 35000*_units.foot  # m
alt_f_cruise = 35000*_units.foot  # m
alt_min_cruise = 35000*_units.foot  # m
alt_max_cruise = 35000*_units.foot  # m
mass_i_cruise = 126000*_units.lb  # kg
mass_f_cruise = 102000*_units.lb  # kg
v_i_cruise = 455.49*_units.knot  # m/s
v_f_cruise = 455.49*_units.knot  # m/s
mach_min_cruise = cruise_mach
mach_max_cruise = cruise_mach
range_i_cruise = 160.3*_units.nautical_mile  # m
range_f_cruise = 3243.9*_units.nautical_mile  # m
t_i_cruise = 26.20*_units.minute  # sec
t_f_cruise = 432.38*_units.minute  # sec
t_duration_cruise = t_f_cruise - t_i_cruise

alt_i_descent = 35000*_units.foot
# final altitude set to 35 to ensure landing is feasible point
# alt_f_descent = 0*_units.foot
alt_f_descent = 35*_units.foot
v_i_descent = 455.49*_units.knot
v_f_descent = 198.44*_units.knot
mach_i_descent = cruise_mach
mach_f_descent = 0.3
mass_i_descent = 102000*_units.pound
mass_f_descent = 101000*_units.pound
range_i_descent = 3243.9*_units.nautical_mile
range_f_descent = 3378.7*_units.nautical_mile
t_i_descent = 432.38*_units.minute
t_f_descent = 461.62*_units.minute
t_duration_descent = t_f_descent - t_i_descent

def load_inputs():
    aviary_inputs = av.get_flops_inputs('N3CC')

    aviary_inputs.set_val(
        Mission.Landing.LIFT_COEFFICIENT_MAX, 2.4, units='unitless')
    aviary_inputs.set_val(
        Mission.Takeoff.LIFT_COEFFICIENT_MAX, 2.0, units='unitless')
    aviary_inputs.set_val(
        Mission.Takeoff.ROLLING_FRICTION_COEFFICIENT, val=.0175,
        units='unitless')

    takeoff_fuel_burned = 577  # lbm
    takeoff_thrust_per_eng = 24555.5  # lbf
    takeoff_L_over_D = 17.35

    aviary_inputs.set_val(
        Mission.Takeoff.FUEL_SIMPLE, takeoff_fuel_burned, units='lbm')
    aviary_inputs.set_val(
        Mission.Takeoff.LIFT_OVER_DRAG, takeoff_L_over_D, units='unitless')
    aviary_inputs.set_val(
        Mission.Design.THRUST_TAKEOFF_PER_ENG, takeoff_thrust_per_eng, units='lbf')

    return aviary_inputs


def add_pre_mission_system(aviary_inputs, prob):
    takeoff_options = av.HeightEnergyTakeoffPhaseBuilder(
        airport_altitude=alt_airport,  # ft
        num_engines=aviary_inputs.get_val(Aircraft.Engine.NUM_ENGINES)
    )

    av.preprocess_crewpayload(aviary_inputs)

    # Upstream static analysis for aero
    prob.model.add_subsystem(
        'pre_mission',
        av.CorePreMission(aviary_options=aviary_inputs,
                       subsystems=av.default_premission_subsystems),
        promotes_inputs=['aircraft:*', 'mission:*'],
        promotes_outputs=['aircraft:*', 'mission:*'])

    takeoff = takeoff_options.build_phase(False)

    prob.model.add_subsystem(
        'takeoff', takeoff, promotes_inputs=['aircraft:*', 'mission:*'],
        promotes_outputs=['mission:*'])

    prob.model.add_subsystem(
        "takeoff_constraints",
        om.ExecComp(
            [
                "takeoff_mass_con=takeoff_mass-climb_start_mass",
                "takeoff_range_con=takeoff_range-climb_start_range",
                "takeoff_vel_con=takeoff_vel-climb_start_vel",
                "takeoff_alt_con=takeoff_alt-climb_start_alt"
            ],
            takeoff_mass_con={'units': 'lbm'}, takeoff_mass={'units': 'lbm'},
            climb_start_mass={'units': 'lbm'},
            takeoff_range_con={'units': 'ft'}, takeoff_range={'units': 'ft'},
            climb_start_range={'units': 'ft'},
            takeoff_vel_con={'units': 'm/s'}, takeoff_vel={'units': 'm/s'},
            climb_start_vel={'units': 'm/s'},
            takeoff_alt_con={'units': 'ft'}, takeoff_alt={'units': 'ft'},
            climb_start_alt={'units': 'ft'}
        ),
        promotes_inputs=[
            ("takeoff_mass", Mission.Takeoff.FINAL_MASS),
            ("takeoff_range", Mission.Takeoff.GROUND_DISTANCE),
            ("takeoff_vel", Mission.Takeoff.FINAL_VELOCITY),
            ("takeoff_alt", Mission.Takeoff.FINAL_ALTITUDE),
        ],
        promotes_outputs=["takeoff_mass_con", "takeoff_range_con",
                          "takeoff_vel_con", "takeoff_alt_con"],
    )

    prob.model.connect(Mission.Takeoff.FINAL_MASS,
                       'traj.climb.initial_states:mass')
    prob.model.connect(Mission.Takeoff.GROUND_DISTANCE,
                       'traj.climb.initial_states:range')
    prob.model.connect(Mission.Takeoff.FINAL_VELOCITY,
                       'traj.climb.initial_states:velocity')
    prob.model.connect(Mission.Takeoff.FINAL_ALTITUDE,
                       'traj.climb.initial_states:altitude')


def add_post_mission_systems(aviary_inputs, prob):
    landing_options = av.HeightEnergyLandingPhaseBuilder(
        ref_wing_area=aviary_inputs.get_val(Aircraft.Wing.AREA, 'ft**2'),
        Cl_max_ldg=aviary_inputs.get_val(Mission.Landing.LIFT_COEFFICIENT_MAX)
    )

    landing = landing_options.build_phase(
        False,
    )

    prob.model.add_subsystem(
        'landing', landing, promotes_inputs=['aircraft:*', 'mission:*'],
        promotes_outputs=['mission:*'])

    prob.model.connect('traj.descent.states:mass',
                       Mission.Landing.TOUCHDOWN_MASS, src_indices=[-1])
    prob.model.connect('traj.descent.states:altitude', Mission.Landing.INITIAL_ALTITUDE,
                       src_indices=[-1])


def set_initial_guesses(prob, traj):
    # have to add this block to recover climb, cruise, and descent phases first
    phase_items = traj._phases.items()
    for idx, (_, phase) in enumerate(phase_items):
        if idx == 0: climb = phase
        elif idx == 1: cruise = phase
        else: descent = phase

    prob.set_val('traj.climb.t_initial', t_i_climb, units='s')
    prob.set_val('traj.climb.t_duration', t_duration_climb, units='s')

    prob.set_val('traj.climb.states:altitude', climb.interp(
        Dynamic.Mission.ALTITUDE, ys=[alt_i_climb, alt_f_climb]), units='m')
    # prob.set_val(
    #     'traj.climb.states:velocity', climb.interp(Dynamic.Mission.VELOCITY, ys=[170, v_f_climb]),
    #     units='m/s')
    prob.set_val('traj.climb.states:velocity', climb.interp(
        Dynamic.Mission.VELOCITY, ys=[v_i_climb, v_f_climb]), units='m/s')
    prob.set_val('traj.climb.states:mass', climb.interp(
        Dynamic.Mission.MASS, ys=[mass_i_climb, mass_f_climb]), units='kg')
    prob.set_val('traj.climb.states:range', climb.interp(
        Dynamic.Mission.RANGE, ys=[range_i_climb, range_f_climb]), units='m')  # nmi

    prob.set_val('traj.climb.controls:velocity_rate',
                 climb.interp(Dynamic.Mission.VELOCITY_RATE, ys=[0.25, 0.0]),
                 units='m/s**2')
    prob.set_val('traj.climb.controls:throttle',
                 climb.interp(Dynamic.Mission.THROTTLE, ys=[1.0, 1.0]),
                 units='unitless')

    prob.set_val('traj.cruise.t_initial', t_i_cruise, units='s')
    prob.set_val('traj.cruise.t_duration', t_duration_cruise, units='s')

    prob.set_val('traj.cruise.states:altitude', cruise.interp(
        Dynamic.Mission.ALTITUDE, ys=[alt_i_cruise, alt_f_cruise]), units='m')
    prob.set_val('traj.cruise.states:velocity', cruise.interp(
        Dynamic.Mission.VELOCITY, ys=[v_i_cruise, v_f_cruise]), units='m/s')
    prob.set_val('traj.cruise.states:mass', cruise.interp(
        Dynamic.Mission.MASS, ys=[mass_i_cruise, mass_f_cruise]), units='kg')
    prob.set_val('traj.cruise.states:range', cruise.interp(
        Dynamic.Mission.RANGE, ys=[range_i_cruise, range_f_cruise]), units='m')  # nmi

    prob.set_val('traj.cruise.controls:velocity_rate',
                 cruise.interp(Dynamic.Mission.VELOCITY_RATE, ys=[0.0, 0.0]),
                 units='m/s**2')
    prob.set_val('traj.cruise.controls:throttle',
                 cruise.interp(Dynamic.Mission.THROTTLE, ys=[0.8, 0.75]),
                 units='unitless')

    prob.set_val('traj.descent.t_initial', t_i_descent, units='s')
    prob.set_val('traj.descent.t_duration', t_duration_descent, units='s')

    prob.set_val('traj.descent.states:altitude', descent.interp(
        Dynamic.Mission.ALTITUDE, ys=[alt_i_descent, alt_f_descent]), units='m')
    prob.set_val('traj.descent.states:velocity', descent.interp(
        Dynamic.Mission.VELOCITY, ys=[v_i_descent, v_f_descent]), units='m/s')
    prob.set_val('traj.descent.states:mass', descent.interp(
        Dynamic.Mission.MASS, ys=[mass_i_descent, mass_f_descent]), units='kg')
    prob.set_val('traj.descent.states:range', descent.interp(
        Dynamic.Mission.RANGE, ys=[range_i_descent, range_f_descent]), units='m')

    prob.set_val('traj.descent.controls:velocity_rate',
                 descent.interp(Dynamic.Mission.VELOCITY_RATE, ys=[-0.25, 0.0]),
                 units='m/s**2')
    prob.set_val('traj.descent.controls:throttle',
                 descent.interp(Dynamic.Mission.THROTTLE, ys=[0.0, 0.0]),
                 units='unitless')


def setup(aviary_inputs, prob):
    prob.model.add_subsystem(
        'input_sink',
        av.VariablesIn(aviary_options=aviary_inputs),
        promotes_inputs=['*'],
        promotes_outputs=['*']
    )

    # suppress warnings:
    # "input variable '...' promoted using '*' was already promoted using 'aircraft:*'
    with warnings.catch_warnings():

        # Set initial default values for all LEAPS aircraft variables.
        warnings.simplefilter("ignore", om.PromotionWarning)
        av.set_aviary_initial_values(prob.model, aviary_inputs)

        warnings.simplefilter("ignore", om.PromotionWarning)
        prob.setup()


def add_objective(prob):
    # This is an example of a overall mission objective
    # create a compound objective that minimizes climb time and maximizes final mass
    # we are maxing final mass b/c we don't have an independent value for fuel_mass yet
    # we are going to normalize these (makign each of the sub-objectives approx = 1 )
    prob.model.add_subsystem(
        "regularization",
        om.ExecComp(
            "reg_objective = - descent_mass_final/60000",
            reg_objective=0.0,
            descent_mass_final={"units": "kg", "shape": 1},
        ),
        promotes_outputs=['reg_objective']
    )
    # connect the final mass from descent into the objective
    prob.model.connect("traj.descent.states:mass",
                       "regularization.descent_mass_final", src_indices=[-1])

    prob.model.add_objective('reg_objective', ref=1)


def add_phases(aviary_inputs, prob):
    ##################
    # Define Phases  #
    ##################

    num_segments_climb = 6
    num_segments_cruise = 1
    num_segments_descent = 5

    climb_seg_ends, _ = dm.utils.lgl.lgl(num_segments_climb + 1)
    descent_seg_ends, _ = dm.utils.lgl.lgl(num_segments_descent + 1)

    transcription_climb = dm.Radau(
        num_segments=num_segments_climb, order=3, compressed=True,
        segment_ends=climb_seg_ends)
    transcription_cruise = dm.Radau(
        num_segments=num_segments_cruise, order=3, compressed=True)
    transcription_descent = dm.Radau(
        num_segments=num_segments_descent, order=3, compressed=True,
        segment_ends=descent_seg_ends)

    climb_options = av.HeightEnergyClimbPhaseBuilder(
        'test_climb',
        user_options=av.AviaryValues({
            'initial_altitude': (alt_i_climb, 'm'),
            'final_altitude': (alt_f_climb, 'm'),
            'initial_mach': (mach_i_climb, 'unitless'),
            'final_mach': (mach_f_climb, 'unitless'),
            'fix_initial': (False, 'unitless'),
            'fix_range': (False, 'unitless'),
            'input_initial': (True, 'unitless'),
        }),
        core_subsystems=av.default_mission_subsystems,
        subsystem_options={'core_aerodynamics': {'method': 'computed'}},
        transcription=transcription_climb,
    )

    cruise_options = av.HeightEnergyCruisePhaseBuilder(
        'test_cruise',
        user_options=av.AviaryValues({
            'min_altitude': (alt_min_cruise, 'm'),
            'max_altitude': (alt_max_cruise, 'm'),
            'min_mach': (mach_min_cruise, 'unitless'),
            'max_mach': (mach_max_cruise, 'unitless'),
            'required_available_climb_rate': (300, 'ft/min'),
            'fix_initial': (False, 'unitless'),
            'fix_final': (False, 'unitless'),
        }),
        core_subsystems=av.default_mission_subsystems,
        subsystem_options={'core_aerodynamics': {'method': 'computed'}},
        transcription=transcription_cruise,
    )

    descent_options = av.HeightEnergyDescentPhaseBuilder(
        'test_descent',
        user_options=av.AviaryValues({
            'final_altitude': (alt_f_descent, 'm'),
            'initial_altitude': (alt_i_descent, 'm'),
            'initial_mach': (mach_i_descent, 'unitless'),
            'final_mach': (mach_f_descent, 'unitless'),
            'fix_initial': (False, 'unitless'),
            'fix_range': (True, 'unitless'),
        }),
        core_subsystems=av.default_mission_subsystems,
        subsystem_options={'core_aerodynamics': {'method': 'computed'}},
        transcription=transcription_descent,
    )

    climb = climb_options.build_phase(aviary_options=aviary_inputs)

    cruise = cruise_options.build_phase(aviary_options=aviary_inputs)

    descent = descent_options.build_phase(aviary_options=aviary_inputs)

    traj = prob.model.add_subsystem('traj', dm.Trajectory())

    climb.set_time_options(
        fix_initial=True, fix_duration=False, units='s',
        duration_bounds=(1, t_duration_climb*2), duration_ref=t_duration_climb)
    cruise.set_time_options(
        fix_initial=False, fix_duration=False, units='s',
        duration_bounds=(1, t_duration_cruise*2), duration_ref=t_duration_cruise)
    descent.set_time_options(
        fix_initial=False, fix_duration=False, units='s',
        duration_bounds=(1, t_duration_descent*2), duration_ref=t_duration_descent)

    traj.add_phase('climb', climb)

    traj.add_phase('cruise', cruise)

    traj.add_phase('descent', descent)

    if use_new_dymos_syntax:  # needed for print_outputs()
        climb.timeseries_options['use_prefix'] = True
        cruise.timeseries_options['use_prefix'] = True
        descent.timeseries_options['use_prefix'] = True

    return traj


def link_phases(aviary_inputs, prob, traj):
    # directly connect phases (strong_couple = True), or use linkage constraints (weak
    # coupling / strong_couple=False)
    strong_couple = False

    traj.link_phases(["climb", "cruise"], ["time", Dynamic.Mission.ALTITUDE,
                     Dynamic.Mission.VELOCITY, Dynamic.Mission.MASS, Dynamic.Mission.RANGE], connected=strong_couple)
    traj.link_phases(["cruise", "descent"], ["time", Dynamic.Mission.ALTITUDE,
                     Dynamic.Mission.VELOCITY, Dynamic.Mission.MASS, Dynamic.Mission.RANGE], connected=strong_couple)

    traj = av.setup_trajectory_params(prob.model, traj, aviary_inputs)

    ##################################
    # Connect in Takeoff             #
    ##################################

    prob.model.connect('traj.climb.states:mass',
                       'takeoff_constraints.climb_start_mass', src_indices=[0])
    prob.model.connect('traj.climb.states:range',
                       'takeoff_constraints.climb_start_range', src_indices=[0])
    prob.model.connect('traj.climb.states:velocity',
                       'takeoff_constraints.climb_start_vel', src_indices=[0])
    prob.model.connect('traj.climb.states:altitude',
                       'takeoff_constraints.climb_start_alt', src_indices=[0])


def run_aviary_problem(prob, sim):
    # Turn off solver printing so that the SNOPT output is readable.
    prob.set_solver_print(level=0)

    dm.run_problem(prob, simulate=sim, make_plots=True, simulate_kwargs={
                   'times_per_seg': 100, 'atol': 1e-9, 'rtol': 1e-9}, 
                   solution_record_file='N3CC_full_mission.db')
    prob.record("final")
    prob.cleanup()


def run_N3CC_full_mission(sim=True):
    ##########################################
    # Build problem                          #
    ##########################################
    prob = om.Problem(model=om.Group())
    #prob.driver = driver

    traj = None

    ##########################################
    # Aircraft Input Variables and Options   #
    ##########################################

    aviary_inputs = load_inputs()

    ##########################################
    # add pre-mission systems                #
    ##########################################

    add_pre_mission_system(aviary_inputs, prob)

    ##########################################
    # add phases                             #
    ##########################################

    traj = add_phases(aviary_inputs, prob)

    ##########################################
    # add post mission system                #
    ##########################################

    add_post_mission_systems(aviary_inputs, prob)

    ##########################################
    # add driver                             #
    ##########################################

    prob.driver = add_driver()

    ##########################################
    # link phases                            #
    ##########################################

    link_phases(aviary_inputs, prob, traj)

    ##########################
    # Add Objective Function #
    ##########################

    add_objective(prob)

    #######
    # setup
    #######

    setup(aviary_inputs, prob)

    ###########################################
    # Intial Settings for States and Controls #
    ###########################################

    set_initial_guesses(prob, traj)

    #####################
    # run aviary problem
    #####################

    run_aviary_problem(prob, sim)

    print_outputs(prob)


def add_driver():
    driver = om.ScipyOptimizeDriver()
    driver.options['optimizer'] = 'SLSQP'
    driver.opt_settings['maxiter'] = 0
    driver.opt_settings['ftol'] = 5.0e-3
    driver.opt_settings['eps'] = 1e-2

    return driver


def print_outputs(prob):
    times_climb = prob.get_val('traj.climb.timeseries.time', units='s')
    altitudes_climb = prob.get_val(
        'traj.climb.timeseries.states:altitude', units='m')
    masses_climb = prob.get_val('traj.climb.timeseries.states:mass', units='kg')
    ranges_climb = prob.get_val('traj.climb.timeseries.states:range', units='m')
    velocities_climb = prob.get_val(
        'traj.climb.timeseries.states:velocity', units='m/s')
    thrusts_climb = prob.get_val('traj.climb.timeseries.thrust_net_total', units='N')
    times_cruise = prob.get_val('traj.cruise.timeseries.time', units='s')
    altitudes_cruise = prob.get_val(
        'traj.cruise.timeseries.states:altitude', units='m')
    masses_cruise = prob.get_val('traj.cruise.timeseries.states:mass', units='kg')
    ranges_cruise = prob.get_val('traj.cruise.timeseries.states:range', units='m')
    velocities_cruise = prob.get_val(
        'traj.cruise.timeseries.states:velocity', units='m/s')
    thrusts_cruise = prob.get_val(
        'traj.cruise.timeseries.thrust_net_total', units='N')
    times_descent = prob.get_val('traj.descent.timeseries.time', units='s')
    altitudes_descent = prob.get_val(
        'traj.descent.timeseries.states:altitude', units='m')
    masses_descent = prob.get_val('traj.descent.timeseries.states:mass', units='kg')
    ranges_descent = prob.get_val('traj.descent.timeseries.states:range', units='m')
    velocities_descent = prob.get_val(
        'traj.descent.timeseries.states:velocity', units='m/s')
    thrusts_descent = prob.get_val(
        'traj.descent.timeseries.thrust_net_total', units='N')

    print("-------------------------------")
    print(f"times_climb: {times_climb[-1]} (s)")
    print(f"altitudes_climb: {altitudes_climb[-1]} (m)")
    print(f"masses_climb: {masses_climb[-1]} (kg)")
    print(f"ranges_climb: {ranges_climb[-1]} (m)")
    print(f"velocities_climb: {velocities_climb[-1]} (m/s)")
    print(f"thrusts_climb: {thrusts_climb[-1]} (N)")
    print(f"times_cruise: {times_cruise[-1]} (s)")
    print(f"altitudes_cruise: {altitudes_cruise[-1]} (m)")
    print(f"masses_cruise: {masses_cruise[-1]} (kg)")
    print(f"ranges_cruise: {ranges_cruise[-1]} (m)")
    print(f"velocities_cruise: {velocities_cruise[-1]} (m/s)")
    print(f"thrusts_cruise: {thrusts_cruise[-1]} (N)")
    print(f"times_descent: {times_descent[-1]} (s)")
    print(f"altitudes_descent: {altitudes_descent[-1]} (m)")
    print(f"masses_descent: {masses_descent[-1]} (kg)")
    print(f"ranges_descent: {ranges_descent[-1]} (m)")
    print(f"velocities_descent: {velocities_descent[-1]} (m/s)")
    print(f"thrusts_descent: {thrusts_descent[-1]} (N)")
    print("-------------------------------")


run_N3CC_full_mission(sim=False)

You see the same outputs.

This model demonstrates the flexibility of level 3. For example, the `load_inputs` function does not load the aircraft model from a `.csv` file but from a Python file using the `get_flops_inputs()` method.

This function not only reads Aviary and mission variables but also builds the engine. More information can be found in `aviary/models/N3CC/N3CC_data.py`.

Note that we can read large single aisle aircraft inputs this way as well:

In [ ]:
aviary_inputs = av.get_flops_inputs('LargeSingleAisle1FLOPS')
aviary_inputs = av.get_flops_inputs('LargeSingleAisle2FLOPS')
aviary_inputs = av.get_flops_inputs('LargeSingleAisle2FLOPSdw')
aviary_inputs = av.get_flops_inputs('LargeSingleAisle2FLOPSalt')

The data files are at:
```
aviary/models/large_single_aisle_1/large_single_aisle_1_FLOPS_data.py
aviary/models/large_single_aisle_2/large_single_aisle_2_FLOPS_data.py
aviary/models/large_single_aisle_2/large_single_aisle_2_detailwing_FLOPS_data.py
aviary/models/large_single_aisle_2/large_single_aisle_2_altwt_FLOPS_data.py
```

respectively.

### Discussing this example in more detail

We move all the code blocks on taxi to `add_pre_mission_system` function because it is how it is done in `methods_for_level2.py`. Similarly, all the code blocks on landing are moved to `add_post_mission_systems` function. Be careful! Generally speaking, not all components can be moved around due to the expected order of execution.

In `aviary/validation_cases/benchmark_tests` folder, there is another N3CC model `test_FLOPS_based_sizing_N3CC.py`. If we had started from that model, you would need to have an `add_design_variables` function. The last function `print_outputs` is added in our model to verify that we didn't alter the original model. It is not a level 2 function but it is a good idea to have this function anyway.

## Summary

We have shown in an example that users have total control over every aspect of the model in level 3. There is one big feature that hasn't been covered: adding external subsystems. We will move on to discuss how to run [Aviary External Subsystem](onboarding_ext_subsystem) next.